<img src='../OUTILS/bandeau_MF.png' align='right' width='100%'/>

<div class="alert alert-info alert-success">
<h3>Elaboration d'image composite avec SATPY- FCI /MTG </h3></div>

## <a id='TOC-TOP'></a>Contenus

<div class="alert alert-block alert-warning">
    
<b>PREREQUIS </b>
    
Ce Notebook ne nécessite pas de prérequis. 
Le kernel "satpy" doit être utilisé

</div>
<hr>

SatPy

Il s'agit d'une bibliothèque Python pour le traitement des données des satellites météorologiques

Satpy permet de lire, manipuler et écrire des données issues d'instruments satellites météorologiques d'observation de la Terre.

Satpy fournit la possibilité de lire les données de différents formats (readers) et de manipuler les paramètres géophysiques de ces différents formats de fichiers.

Satpy fournit également des interfaces pour créer des images RVB (rouge/vert/bleu) qui combinent les données de plusieurs bandes d'instruments ou produits.

Diverses corrections atmosphériques et améliorations visuelles sont fournies pour améliorer le rendu et la qualité des images de sortie. Les données de sortie peuvent être écrites dans plusieurs formats de fichiers tels que les fichiers NetCDF, PNG et GeoTIFF.

Satpy permet également aux utilisateurs de rééchantillonner les données sur des grilles géographiques projetées (zones).

Pour obtenir de la documentation sur Satpy, quelques exemples, et un tutoriel de démarrage rapide : https://satpy.readthedocs.io/en/latest/index.html

<div class="alert alert-info" role="alert">

## <a id='section1'></a>1.Tout d'abord, il faut procéder à l'importation des librairies nécessaires.

</div>

In [ ]:
from satpy.scene import Scene
from satpy import find_files_and_readers
from datetime import datetime
import sys
from pyresample.geometry import AreaDefinition
import numpy as np
import os
from PIL import Image
import subprocess

In [ ]:
os.system("echo $PATH")

### Données d'entrée : fichiers chunk mtg (40)

In [ ]:
input = '../../MF_DATA/MTG/chunk_decomp1/'

In [ ]:
download_dir = os.path.join(os.getcwd(), "../RESULTATS")
os.makedirs(download_dir, exist_ok=True)

In [ ]:
output = '../RESULTATS'

### Choix de la RGB à produire, parmi notamment:
airmass ash cimss_cloud_type cloud_phase cloud_phase_distinction cloud_phase_distinction_raw cloud_phase_raw cloudtop convection day_microphysics dust fog geo_color green_snow ir108_3d ir_cloud_day natural_color ndvi_hybrid_green night_fog night_microphysics rocket_plume_day rocket_plume_night true_color true_color_reproduction 

In [ ]:
composite_name = "dust"

### Définition de la date et heure de la composite à créer
(202401091200 disponible par défaut)

In [ ]:
yyyy=int('2024')
mm=int('01')
dd=int('09')
hh_debut=int('12')
min_debut=int('00')
hh_fin=int('12')
min_fin=int('10')

In [ ]:
### Définition du reader

In [ ]:
reader_to_use = "fci_l1c_nc"

In [ ]:
filename = (output + '/RGB_sortie.tif' )
filename_image_out = (output + '/RGB_' + composite_name + '_min' )
filename_image_zoom_out = (output + '/RGB_' + composite_name + '_zoom_min' )

In [ ]:
myfiles = find_files_and_readers(base_dir=input,
                                 start_time=datetime(yyyy,mm,dd,hh_debut,min_debut),
                                 end_time=datetime(yyyy,mm,dd,hh_fin,min_fin),
                                 reader=reader_to_use)

In [ ]:
!gdalinfo ../../MF_DATA/MTG/chunk_decomp1/W_XX-EUMETSAT-Darmstadt,IMG+SAT,MTI1+FCI-1C-RRAD-FDHSI-FD--CHK-BODY--DIS-NC4E_C_EUMT_20240109121040_IDPFI_OPE_20240109120908_20240109120924_N_JLS_C_0073_0040.nc

In [ ]:
!gdalinfo NETCDF:"../../MF_DATA/MTG/chunk_decomp1/W_XX-EUMETSAT-Darmstadt,IMG+SAT,MTI1+FCI-1C-RRAD-FDHSI-FD--CHK-BODY--DIS-NC4E_C_EUMT_20240109121040_IDPFI_OPE_20240109120908_20240109120924_N_JLS_C_0073_0040.nc":/data/vis_04/measured/pixel_quality

In [ ]:
scn = Scene(filenames=myfiles)

### Liste de toutes les composites disponibles

In [ ]:
print(scn.available_composite_names())

### Liste de tous les dataset disponibles

In [ ]:
print(scn.available_dataset_names())

### Creation du produit

In [ ]:
scn.load([composite_name], upper_right_corner='NE')

In [ ]:
natscn = scn.resample(scn.coarsest_area(), resampler='nearest')

In [ ]:
natscn.save_dataset(composite_name, filename=filename)

### Défnition de la taille de l'image globe (pixels)

In [ ]:
taille_redim = 800  # 5568x5568 par défaut

In [ ]:
# Construire la commande gdalwarp
cmd1 = [
    'gdalwarp',
    '-ts', str(taille_redim), str(taille_redim),     # Option pour spécifier la taille en pixels (taille_redimxtaille_redim)
    '-overwrite',              # Option pour écraser le fichier de sortie s'il existe
    filename,
    filename_image_out + '.tif'
]
#correspond par exemple à la commande: gdalwarp -ts 800 800 RGB_sortie.tif RGB_min.tif

In [ ]:
subprocess.run(cmd1)

### Rappel : nom du fichier de sortie

In [ ]:
filename_image_out

In [ ]:
# Chemins vers les fichiers shapefile contenant les frontieres des pays
input_shapefile = '../OUTILS/boundary/world-administrative-boundaries.shp'  # Chemin vers le shapefile

### Ajout des frontières

In [ ]:
# Définir la commande gdal_rasterize
cmd2 = [
    'gdal_rasterize',
    '-b', '1' ,
    '-burn', '255',  # Valeur de brûlure
    '-b', '2' ,
    '-burn', '255',  # Valeur de brûlure
    '-b', '3' ,
    '-burn', '255',  # Valeur de brûlure
    '-l', 'world-administrative-boundaries',  # Nom de la couche
    input_shapefile,  # Chemin vers le shapefile
    filename_image_out + ".tif"  # Chemin vers l'image raster de sortie
]

#Correspondance de la commande en 1 ligne : gdal_rasterize -b 1 -burn 255 -b 2 -burn 255 -b 3 -burn 255 -l world-administrative-boundaries ../OUTILS/boundary/world-administrative-boundaries.shp RGB_min.tif

In [ ]:
# Exécuter la commande gdal_rasterize
#subprocess.run(cmd1) # (montre les erreurs "Point outside of projection domain")
with open(os.devnull, 'w') as devnull:
    subprocess.run(cmd2, stderr=devnull)

### Affichage de l'image globe dans le jupyter notebook

In [ ]:
im = Image.open(filename_image_out + '.tif')
display(im)

### Création d'une image à partir d'une zone de découpe 

In [ ]:
### Définition de la zone, et redimmensionnement de l'image 

In [ ]:
#exemple nord-ouest Afrique = 40,8,-23,15
coord_nord = 40
coord_sud = 8
coord_ouest = -23
coord_est = 15


In [ ]:
# Facteur de mise à l'échelle
scale_factor = 0.04  # 0.04 par défaut

In [ ]:
# Construire la commande gdalwarp pour la découpe
cmd3 = [
    'gdalwarp',
    '-t_srs', 'EPSG:4326',     # Option pour spécifier la projection
    '-te', str(coord_ouest), str(coord_est), str(coord_sud), str(coord_nord),  # Option pour spécifier l'étendue
     '-tr', str(scale_factor), str(scale_factor),  # Facteur de mise à l'échelle en x et y
    '-overwrite',              # Option pour écraser le fichier de sortie s'il existe
    filename,
    filename_image_zoom_out + '.tif'
]

In [ ]:
subprocess.run(cmd3)

In [ ]:
#im2 = Image.open(filename_image_zoom_out + '.tif')
#display(im2)

In [ ]:
#sauvegarde du tif d'origine (gdal_rasterize écrase)
!cp {filename_image_zoom_out + ".tif"} {filename_image_zoom_out + "_contour.tif"}

#### Ajout des frontières

In [ ]:
# Définir la commande gdal_rasterize
cmd4 = [
    'gdal_rasterize',
    '-b', '1' ,
    '-burn', '255',  # Valeur de brûlure
    '-b', '2' ,
    '-burn', '255',  # Valeur de brûlure
    '-b', '3' ,
    '-burn', '255',  # Valeur de brûlure
    '-l', 'world-administrative-boundaries',  # Nom de la couche
    input_shapefile,  # Chemin vers le shapefile
    filename_image_zoom_out + "_contour.tif"  # Chemin vers l'image raster de sortie
]

In [ ]:
subprocess.run(cmd4)

In [ ]:
im2 = Image.open(filename_image_zoom_out + '_contour.tif')
display(im2)

### Reprojection de l'image

#### Définition de la reprojection de l'image à reprojeter

In [ ]:
#Modification des coordonnées

#projection = "+proj=ortho +lat_0=0 +lon_0=0"
projection = "+proj=ortho +lat_0=45 +lon_0=5"

In [ ]:
filename_image_reproj_zoom_out = (output + '/RGB_' + composite_name + 'ortho_zoom_min' )

In [ ]:
filename_image_reproj_zoom_out

In [ ]:
# Construire la commande gdalwarp
cmd5 = [
    'gdalwarp',
    '-t_srs', projection,  # Système de référence cible
    '-overwrite',              # Option pour écraser le fichier de sortie s'il existe
    filename_image_zoom_out + '_contour.tif',
    filename_image_reproj_zoom_out + '.tif'
]

In [ ]:
subprocess.run(cmd5)

In [ ]:
im3 = Image.open(filename_image_reproj_zoom_out + '.tif')
display(im3)

### Nettoyage

In [ ]:
!rm {output}/*xml {output}/RGB_sortie.tif